In [17]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_single_line_avr(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    # global variable used to save the state of this processing for later use
    global m_left, m_right
    global avr_left_x, avr_left_y, avr_right_x, avr_right_y
    global l_low, l_high, r_low, r_high
    global first_time
    
    xMax = img.shape[1]
    yMax = img.shape[0]

    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

    no_left_lines = 0
    no_right_lines = 0
    
    sum_slope_left = 0
    sum_slope_right = 0
    
    sum_left_x = 0
    sum_left_y = 0

    sum_right_x = 0
    sum_right_y = 0
    
    m_left_w = 0
    m_right_w = 0
    
    # go over all lines detected by Canny edge detection
    for line in lines:
        for x1,y1,x2,y2 in line:
            # find the slope of the line
            m = (y2-y1)/(x2-x1)
            # create a weight for the slope, the closer the line to the buttom edge of the image, the higher the weight
            # this is true as the camera has a better accuracy (and so does Canny) for lines closer to the buttom edge
            w_local = 1/(yMax - max(y1, y2) + 1)
            # filter out lines that do not conform to the expected slope of the lane lines
            if abs(m) > 0.1:
                # for left and right lane lines average the mid-point and the slope
                if m < 0: #left lane
                    sum_slope_left += m*w_local
                    m_left_w += w_local
                    no_left_lines += 1
                    sum_left_x += x1 + x2
                    sum_left_y += y1 + y2
                else: #right lane
                    sum_slope_right += m*w_local
                    m_right_w += w_local
                    no_right_lines += 1
                    sum_right_x += x1 + x2
                    sum_right_y += y1 + y2
    
    # if canny has failed to capture any lane lines, just use the previous info
    # otherwise take the average for a point on the line as well as the slope of the line
    if no_left_lines == 0:
        m_left_local = 0.1*m_left
        avr_left_x_local = 0.1*avr_left_x
        avr_left_y_local = 0.1*avr_left_y
    else:
        m_left_local = sum_slope_left/m_left_w
        avr_left_x_local = sum_left_x/(2*no_left_lines)
        avr_left_y_local = sum_left_y/(2*no_left_lines)

    if no_right_lines == 0:
        m_right_local = 0.1*m_right
        avr_right_x_local = 0.1*avr_right_x
        avr_right_y_local = 0.1*avr_right_y
    else:    
        m_right_local = sum_slope_right/m_right_w
        avr_right_x_local = sum_right_x/(2*no_right_lines)
        avr_right_y_local = sum_right_y/(2*no_right_lines)
    
    # use the previous processing information to have a more stable processing
    if first_time == 1:
        m_left = m_left_local
        m_right = m_right_local
        avr_left_x = avr_left_x_local
        avr_left_y = avr_left_y_local
        avr_right_x = avr_right_x_local
        avr_right_y = avr_right_y_local
    else:
        m_left = 0.9*m_left + 0.1*m_left_local
        m_right = 0.9*m_right + 0.1*m_right_local
        avr_left_x = 0.9*avr_left_x + 0.1*avr_left_x_local
        avr_left_y = 0.9*avr_left_y + 0.1*avr_left_y_local
        avr_right_x = 0.9*avr_right_x + 0.1*avr_right_x_local
        avr_right_y = 0.9*avr_right_y + 0.1*avr_right_y_local
            
    b_left = avr_left_y - m_left*avr_left_x
    b_right = avr_right_y - m_right*avr_right_x

    lower_y = yMax
    lower_left_x = int(round((lower_y - b_left)/m_left))
    lower_right_x = int(round((lower_y - b_right)/m_right))
    
    upper_y = int(round(yMax/2 + yMax/10))
    upper_left_x = int(round((upper_y - b_left)/m_left))
    upper_right_x = int(round((upper_y - b_right)/m_right))
    
    # use these updated points for the region restriction for canny
    l_low = lower_left_x - 10
    l_high = upper_left_x - 10
    r_low = lower_right_x + 10
    r_high = upper_right_x + 10
    
    blank_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    cv2.line(blank_img, (lower_left_x, lower_y), (upper_left_x, upper_y), [255, 0, 0], 5)
    cv2.line(blank_img, (lower_right_x, lower_y), (upper_right_x, upper_y), [255, 0, 0], 5)

    return blank_img

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def pipeline(img):
    global first_time
    
    xMax = img.shape[1]
    yMax = img.shape[0]

    #convert the image to greyscale
    grey_image = grayscale(img)

    #denoise the image using gaussian filter
    blurred_image = gaussian_blur(grey_image, 5)

    #find the edges
    edges = canny(blurred_image, 100, 150)

    #get the region of interest
    if first_time == 1:
        polygon = np.array([ [xMax/10,yMax], [xMax/2 - xMax/20,yMax/2 + yMax/10], [xMax/2 + xMax/20,yMax/2 + yMax/10], [9*xMax/10,yMax] ], np.int32)
    else:
        polygon = np.array([ [l_low,yMax], [l_high,yMax/2 + yMax/10], [r_high,yMax/2 + yMax/10], [r_low,yMax] ], np.int32)
    masked_image = region_of_interest(edges, [polygon] )

    #hough transform application
    rho = 1
    theta = np.pi/180
    threshold = 40
    min_line_len = 1
    max_line_gap = 5
    # hough_image = hough_lines(masked_image, rho, theta, threshold, min_line_len, max_line_gap)
    # plt.imshow(hough_image, cmap='gray')
#     hough_image = hough_single_line(masked_image, rho, theta, threshold, min_line_len, max_line_gap)
    hough_image = hough_single_line_avr(masked_image, rho, theta, threshold, min_line_len, max_line_gap)

    final_image = weighted_img(hough_image, img, α=0.8, β=1., λ=0.)
    
    first_time = 0

    return final_image

def process_image(image):

    return pipeline(image)

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

#reading in an image
# original_image = mpimg.imread('test_images/solidWhiteRight.jpg')
# final_image = pipeline(original_image)
# plt.imshow(final_image, cmap='gray')

# initialize the global variables used in the processing
first_time = 1
m_left = 0
m_right = 0
avr_left_x = 0
avr_left_y = 0
avr_right_x = 0
avr_right_y = 0
l_low = 0
l_high = 0
r_low = 0
r_high = 0

white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

# initialize the global variables used in the processing
first_time = 1
m_left = 0
m_right = 0
avr_left_x = 0
avr_left_y = 0
avr_right_x = 0
avr_right_y = 0
l_low = 0
l_high = 0
r_low = 0
r_high = 0

white_output = 'yellow.mp4'
clip1 = VideoFileClip("solidYellowLeft.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)


## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!



Something else which is useful is to normalize the image. This is specifically useful when we have shaddows in the video which interferes with the span of the greyscale image. I would probably find the min and max of the greyscale frame and expand that to include the full range of 0-255. With this the canny edge detection results are more accurate and the thresholds already in place will act much better.